In [5]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_csv('stock_data.csv')

In [7]:
new_df=pd.DataFrame()
new_df['Close']=data['Close']
data=new_df

In [8]:
print(data['Close'].isnull().sum())

306


In [9]:
data.dropna(inplace=True)
print(data.isnull().sum())

Close    0
dtype: int64


In [10]:
from collections import Counter
a=[]
for i in data['Close']:
  if(i!='Close'):
    a.append(i)
print(Counter(a))
new_df=pd.DataFrame()
new_df['Close']=a
data=new_df

Counter({818.200012: 500, 127.25: 17, 156.5: 16, 155.550003: 15, 120.0: 15, 363.700012: 15, 74.250015: 15, 137.75: 14, 135.0: 14, 117.050003: 13, 115.650002: 13, 129.75: 13, 120.900002: 13, 167.399994: 13, 149.899994: 13, 157.25: 13, 153.100006: 13, 123.550003: 13, 153.5: 13, 152.050003: 13, 147.5: 13, 275.5: 13, 133.899994: 13, 121.75: 13, 343.799988: 12, 116.800003: 12, 118.25: 12, 118.5: 12, 136.5: 12, 255.100006: 12, 350.549988: 12, 146.699997: 12, 143.899994: 12, 117.75: 12, 148.300003: 12, 154.850006: 12, 142.899994: 12, 117.0: 12, 143.25: 12, 143.300003: 12, 142.0: 12, 159.699997: 12, 181.699997: 12, 121.199997: 12, 175.75: 12, 353.600006: 11, 120.550003: 11, 127.699997: 11, 116.75: 11, 128.5: 11, 126.5: 11, 124.050003: 11, 124.300003: 11, 117.300003: 11, 117.5: 11, 288.899994: 11, 290.0: 11, 281.0: 11, 326.0: 11, 492.0: 11, 382.799988: 11, 313.149994: 11, 382.549988: 11, 314.0: 11, 332.200012: 11, 267.5: 11, 499.950012: 11, 569.5: 11, 767.400024: 11, 300.649994: 11, 143.75: 11,

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(data)

MinMaxScaler()

In [12]:
new_data=scaler.transform(data)
data=pd.DataFrame(new_data,columns=data.columns)

In [13]:
data['Close'] = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

# Define the training and testing datasets
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps=60
X_train, y_train = create_dataset(train_data[['Close']], train_data[['Close']], time_steps)
X_test, y_test = create_dataset(test_data[['Close']], test_data[['Close']], time_steps)


In [14]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Load the data
#data = pd.read_csv('nifty50.csv')

# Preprocess the data
#scaler = MinMaxScaler()
data['Close'] = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

# Define the training and testing datasets
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps=60
X_train, y_train = create_dataset(train_data[['Close']], train_data[['Close']], time_steps)
X_test, y_test = create_dataset(test_data[['Close']], test_data[['Close']], time_steps)

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.GRU(units=128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GRU(units=64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=64)

Epoch 1/3
1756/1756 [==============================] - 224s 126ms/step - loss: 1.7167e-04
Epoch 2/3
1756/1756 [==============================] - 223s 127ms/step - loss: 1.0803e-04
Epoch 3/3
1756/1756 [==============================] - 235s 134ms/step - loss: 1.0620e-04


In [15]:
model.save('gru_model.h5')

In [1]:
import pickle
with open('gru_overall','rb') as f:
  mo=pickle.load(f)

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-07-05 14:34:24         2921
metadata.json                                  2023-07-05 14:34:24           64
variables.h5                                   2023-07-05 14:34:24      1080248
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...layers\gru
......vars
...layers\gru\cell
......vars
.........0
.........1
.........2
...layers\gru_1
......vars
...layers\gru_1\cell
......vars
.........0
.........1
.........2
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars


In [16]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import datetime
import time
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
def inverse(a,b):
  m1= b.min()
  m2 = b.max()
  return a*(m2-m1) + m1

def mean_squared_error(y_predicted,y_actual):
  return ((y_predicted-y_actual)**2).sum()/(len(y_predicted))

print('Following are the commands you may enter :')
while(True):
  print("0. If you want to terminate")
  print("1. If you want to know the actual stock price")
  print("2. If you want to know a range of stock price")
  input_from_user=int(input("Enter the command : "))
  if(input_from_user==0):
    break
  elif(input_from_user==1):
    company_name = input('Enter company name : ')
    with open('gru_overall','rb') as f:
      mo = pickle.load(f)
    for layer in mo.layers:
      layer.trainable = False
    ticker_symbol = company_name.upper()+".NS"
    days_back = 15
    interval='15m'
    end_date = datetime.datetime.now().strftime('%Y-%m-%d')
    start_date = (datetime.datetime.now() - datetime.timedelta(days=days_back)).strftime('%Y-%m-%d')
    company_data=yf.download(ticker_symbol, start=start_date, end=end_date,interval=interval)['Close']
    df=scaler.fit_transform(np.array(company_data).reshape(-1,1))
    new_data=pd.DataFrame()
    df=np.array(df)
    l = []
    for i in range(df.shape[0]):
      l.append(df[i][0])
    new_data['Close']=l
    new_data=pd.DataFrame(new_data,columns=["Close"])
    def create_dataset(X, y, time_steps=1):
      Xs, ys = [], []
      for i in range(len(X) - time_steps):
          Xs.append(X.iloc[i:(i + time_steps)].values)
          ys.append(y.iloc[i + time_steps])
      return np.array(Xs), np.array(ys)
    X_test,y_test = create_dataset(new_data[['Close']], new_data[['Close']],60)
    mo.fit(X_test, y_test,epochs=10, batch_size=8)
    hh = (mo.predict(X_test))
    pred=inverse(hh,company_data)
    p=pred.flatten()
    y_predicted=np.array(p)
    comp_data=list(company_data)
    y_actual=np.array(comp_data[60:])
    print()
    print()
    print(f'Mean Squared Error for {company_name} is {mean_squared_error(y_predicted,y_actual)}')
    print()
    print()
    new_x_test=np.array(new_data[len(company_data) - 60:]).reshape(1,60,1)
    pred=mo.predict(new_x_test)
    next_predicted=inverse(pred,company_data)[0][0]
    print()
    print()
    print("Next day's prediction: ",next_predicted)
    print()
    print()

  elif(input_from_user==2):
    company_name = input('Enter company name : ')
    predictions=[]
    for i in range(5):
      with open('gru_overall','rb') as f:
        mo = pickle.load(f)
      for layer in mo.layers:
        layer.trainable = False
      ticker_symbol = company_name.upper()+".NS"
      days_back = 15
      interval='15m'
      end_date = datetime.datetime.now().strftime('%Y-%m-%d')
      start_date = (datetime.datetime.now() - datetime.timedelta(days=days_back)).strftime('%Y-%m-%d')
      company_data=yf.download(ticker_symbol, start=start_date, end=end_date,interval=interval)['Close']
      df=scaler.fit_transform(np.array(company_data).reshape(-1,1))
      new_data=pd.DataFrame()
      df=np.array(df)
      l = []
      for i in range(df.shape[0]):
        l.append(df[i][0])
      new_data['Close']=l
      new_data=pd.DataFrame(new_data,columns=["Close"])

      def create_dataset(X, y, time_steps=1):
        Xs, ys = [], []
        for i in range(len(X) - time_steps):
            Xs.append(X.iloc[i:(i + time_steps)].values)
            ys.append(y.iloc[i + time_steps])
        return np.array(Xs), np.array(ys)
      X_test,y_test = create_dataset(new_data[['Close']], new_data[['Close']],60)
      mo.fit(X_test, y_test,epochs=10,batch_size=8,verbose=True)
      hh = (mo.predict(X_test))
      pred=inverse(hh,company_data)
      p=pred.flatten()
      y_predicted=np.array(p)
      comp_data=list(company_data)
      y_actual=np.array(comp_data[60:])
      print()
      print()
      print(f'Mean Squared Error for {company_name} is {mean_squared_error(y_predicted,y_actual)}')
      print()
      print()
      new_x_test=np.array(new_data[len(company_data)-60:]).reshape(1,60,1)
      pred=mo.predict(new_x_test)
      next_predicted=inverse(pred,company_data)[0][0]
      predictions.append(next_predicted)
    predictions=np.array(predictions)
    mean=predictions.mean()
    standard_deviation=predictions.std()
    print(f'Stock Price is supposed to be in range : [ {mean-standard_deviation} , {mean+standard_deviation} ]')



Following are the commands you may enter :
0. If you want to terminate
1. If you want to know the actual stock price
2. If you want to know a range of stock price


ValueError: invalid literal for int() with base 10: ''